In [49]:
%load_ext autoreload
%autoreload 2
import torch
from torch import nn
import torchvision as tv
import torchvision.transforms as tfms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import hw_3
import hw_3_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
 
train_transform = tfms.Compose([tfms.Resize((256, 256)), tfms.ToTensor(), tfms.Normalize(mean, std)])
val_transform = tfms.Compose([tfms.Resize((256, 256)), tfms.ToTensor(), tfms.Normalize(mean, std)])
 
# train_dataset = tv.datasets.ImageFolder(f'/local/temporary/vir/hw3/train', transform=train_transform)
# val_dataset = tv.datasets.ImageFolder(f'/local/temporary/vir/hw3/val', transform=val_transform)
train_dataset = tv.datasets.ImageFolder(f'vir/hw3/train', transform=train_transform)
val_dataset = tv.datasets.ImageFolder(f'vir/hw3/val', transform=val_transform)

In [51]:
BATCH_SIZE = 4
train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=1, shuffle=False)

In [4]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# img1_hehehe = next(iter(train_dataloader))
# imshow(img1_hehehe[0][1])
# print((img1_hehehe[0][0]).shape)

In [44]:
# device = hw_3_utils.get_device_mps()
# print(device)
# model = tv.models.resnet18(weights=tv.models.ResNet18_Weights.IMAGENET1K_V1)
# for param in model.parameters():
#     param.requires_grad = False
# model.fc = nn.Linear(model.fc.in_features, 50)
# model = model.to(device)


In [119]:
device = hw_3_utils.get_device_mps()
print(device)
model = hw_3.NotSoMyResNet(block=hw_3.Block, layers=[2, 2, 2, 2], num_classes=50)
# model.load_state_dict(torch.load('weights_my_res_net.pts'))
model = model.to(device)

mps


In [120]:
loss_fn = nn.CrossEntropyLoss()
# opt = SGD(model.parameters(), lr=0.1, )
# opt = torch.optim.SGD(model.parameters(), lr=1.0e-3, momentum=0.9)
opt = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

In [121]:
def train_epoch(model, train_loader, loss_fn, optimizer, device):
    model.train()
    loss_sum = 0
    for idx, batch in enumerate(train_loader):
        data, labels = batch[0].to(device), batch[1].to(device)

        output = model(data)
        print("wkrfre")
        loss = loss_fn(output, labels) / 2
        loss.backward()

        if idx % 2 == 0:
            optimizer.step()
            optimizer.zero_grad()
        
        loss_sum += loss.item()

        if idx % 100 == 0:
            print(loss.item())
        
    loss = loss_sum / (idx+1)
    return model, loss

In [122]:
epoch = 1
for i in range(epoch):
    print(f"epoch {i + 1}/{epoch}")
    _, loss = train_epoch(model, train_loader=train_dataloader, loss_fn=loss_fn, optimizer=opt, device=device)
    print(f"\rTrain loss: {loss:.04f}\r", end="")

epoch 1/1
wkrfre
1.9655070304870605
wkrfre
wkrfre
wkrfre
wkrfre
wkrfre
wkrfre
wkrfre
wkrfre
wkrfre
wkrfre
wkrfre
wkrfre


KeyboardInterrupt: 

: 

In [27]:
# torch.save(model.state_dict(), 'weights_resnet18_torch.pts')
# torch.save(model.state_dict(), 'weights_resnet18_only_head.pts')
# torch.save(model.state_dict(), 'weights_resnet18_only_head2.pts')
torch.save(model.state_dict(), 'weights_my_res_net_2.pts', _use_new_zipfile_serialization=False)


In [11]:
def test_model(model, test_loader, device):
    model.eval()
    acc = 0
    # START FILL
    # device='cpu'
    with torch.no_grad():
        for idx, batch in enumerate(test_loader):
            data, labels = batch[0].to(device), batch[1].to(device)
            output = model(data)
            acc += accuracy(output, labels, dim=1)
    
    acc /= (idx + 1)
    # END FILL
    print(f'{acc * 100:.1f}%')
    
def accuracy(prediction, labels_batch, dim=1):
    pred = torch.topk(prediction, 3).indices
    labels_batch = labels_batch.cpu()
    acc = 0
    for i, x in enumerate(labels_batch):
        acc += (x in pred)


    # pred = torch.argmax(prediction.cpu(), dim=1)
    # acc = (pred == labels_batch.cpu()).float().mean().numpy()

    return acc
    
    # return acc / labels_batch.shape[0]

In [29]:
test_model(model, test_loader=val_dataloader, device=device)


68.8%


In [15]:
# print((tv.models.resnet18(weights=tv.models.ResNet18_Weights.IMAGENET1K_V1)).modules)
x = torch.randn(1, 4)
y = x.view(x.size(0), -1)
y.size()
# x.size(0)

torch.Size([1, 4])

In [19]:
# net = hw_3.NotSoMyResNet()
# net.load_state_dict(torch.load('weights_my_res_net.pts'))
# net = net.to(device)

In [20]:
test_model(net, test_loader=val_dataloader, device=device)


53.8%


In [87]:
np.round(1.23423423, 2)

1.23